In [1]:
import xarray as xr
import iris
import numpy as np

iris.FUTURE.netcdf_promote = True

In [2]:
import os
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from scipy.misc import toimage

def list_mogreps_uk(folder, years, months, days, hours, fcsts):
    fs = ['prods_op_mogreps-uk_{:04d}{:02d}{:02d}_{:02d}_00_{:03d}.nc'.format(year, month, day, hour, fcst)
            for year in years for month in months for day in days for hour in hours for fcst in fcsts]
    fs = [os.path.join(folder,f) for f in fs]
    return fs

class PrecipDataset(Dataset):
    def __init__(self, array, transform=None):
        self.array = array
        self.transform = transform

    def __len__(self):
        return len(self.array)

    def __getitem__(self, idx):
        try:
            precip = self.array[idx]

            scale_factor = 4
            low_res = np.add.reduceat(np.add.reduceat(precip.data, 
                                                      list(range(0, precip.data.shape[0], scale_factor))),
                                      list(range(0, precip.data.shape[1], scale_factor)), 
                                      axis=1) / scale_factor ** 2

            img = toimage(low_res.data)

            if self.transform:
                img = self.transform(img)

            return img, img
        except:
            return self.__getitem__(np.random.randint(0, len(self.array)))

In [3]:
arrays = np.load('data/cloud_data.npz')

input_data = arrays['data']
test_data = arrays['test']

In [4]:
precip = PrecipDataset(input_data)

In [5]:
precip[5]

(<PIL.Image.Image image mode=L size=106x137 at 0x7F9C3B3DE6D8>,
 <PIL.Image.Image image mode=L size=106x137 at 0x7F9C3B3DE6D8>)

In [6]:
106*137

14522

In [7]:
from torchvision import datasets, transforms
precip_train = PrecipDataset(input_data,
                             transform=transforms.ToTensor())

precip_test = PrecipDataset(test_data,
                            transform=transforms.ToTensor())

# precip_train = list(precip_train)
# precip_test = list(precip_test)

In [8]:
14522 * 3200

46470400

In [9]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

args_dict = {'batch_size': 5,
             'epochs': 100,
             'cuda': True,
             'seed': 1,
             'log_interval': 10}
args = type('test', (object,), {})()
args.__dict__.update(args_dict)


torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

train_loader = torch.utils.data.DataLoader(precip_train,
                                           batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(precip_test,
                                          batch_size=args.batch_size, shuffle=True, **kwargs)

N = 14522
# n2 = int(N/2)
# n3 = int(N/9)
n2 = 3200
n3 = 10

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(N, n2)
        self.fc21 = nn.Linear(n2, n3)
        self.fc22 = nn.Linear(n2, n3)
        self.fc3 = nn.Linear(n3, n2)
        self.fc4 = nn.Linear(n2, N)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, N))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
if args.cuda:
    model.cuda()


def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, N))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # Normalise by same number of elements as in reconstruction
    KLD /= args.batch_size * N

    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        print(batch_idx)
        data = Variable(data)
        if args.cuda:
            data = data.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        print(i)
        if args.cuda:
            data = data.cuda()
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]
        if i == 0:
            n = min(data.size(0), 8)
            comparison = torch.cat([data[:n],
                                  recon_batch.view(args.batch_size, 1, 137, 106)[:n]])
            save_image(comparison.data.cpu(),
                     'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [10]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test(epoch)
    sample = Variable(torch.randn(64, n3))
    if args.cuda:
        sample = sample.cuda()
    sample = model.decode(sample).cpu()
    save_image(sample.data.view(64, 1, 137, 106),
               'results/sample_' + str(epoch) + '.png')

0
Train Epoch: 1 [0/504 (0%)]	Loss: 0.140058
1
2
3
4
5
6
7
8
9
10
Train Epoch: 1 [50/504 (10%)]	Loss: 0.200654
11
12
13
14
15
16
17
18
19
20
Train Epoch: 1 [100/504 (20%)]	Loss: 0.143990
21
22
23
24
25
26
27
28
29
30
Train Epoch: 1 [150/504 (30%)]	Loss: 0.173469
31
32
33
34
35
36
37
38
39
40
Train Epoch: 1 [200/504 (40%)]	Loss: 0.146785
41
42
43
44
45
46
47
48
49
50
Train Epoch: 1 [250/504 (50%)]	Loss: 0.138397
51
52
53
54
55
56
57
58
59
60
Train Epoch: 1 [300/504 (59%)]	Loss: 0.135972
61
62
63
64
65
66
67
68
69
70
Train Epoch: 1 [350/504 (69%)]	Loss: 0.137675
71
72
73
74
75
76
77
78
79
80
Train Epoch: 1 [400/504 (79%)]	Loss: 0.131364
81
82
83
84
85
86
87
88
89
90
Train Epoch: 1 [450/504 (89%)]	Loss: 0.125415
91
92
93
94
95
96
97
98
99
100
Train Epoch: 1 [400/504 (99%)]	Loss: 0.160938
====> Epoch: 1 Average loss: 0.1621
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1275
0
Train Epoch: 2 [0/504 (0%)]	Loss: 0.130756
1
2
3
4
5
6
7
8
9
10
Train Epoch:

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1166
0
Train Epoch: 10 [0/504 (0%)]	Loss: 0.098980
1
2
3
4
5
6
7
8
9
10
Train Epoch: 10 [50/504 (10%)]	Loss: 0.105965
11
12
13
14
15
16
17
18
19
20
Train Epoch: 10 [100/504 (20%)]	Loss: 0.102361
21
22
23
24
25
26
27
28
29
30
Train Epoch: 10 [150/504 (30%)]	Loss: 0.101442
31
32
33
34
35
36
37
38
39
40
Train Epoch: 10 [200/504 (40%)]	Loss: 0.100658
41
42
43
44
45
46
47
48
49
50
Train Epoch: 10 [250/504 (50%)]	Loss: 0.111956
51
52
53
54
55
56
57
58
59
60
Train Epoch: 10 [300/504 (59%)]	Loss: 0.096109
61
62
63
64
65
66
67
68
69
70
Train Epoch: 10 [350/504 (69%)]	Loss: 0.096760
71
72
73
74
75
76
77
78
79
80
Train Epoch: 10 [400/504 (79%)]	Loss: 0.118824
81
82
83
84
85
86
87
88
89
90
Train Epoch: 10 [450/504 (89%)]	Loss: 0.101914
91
92
93
94
95
96
97
98
99
100
Train Epoch: 10 [400/504 (99%)]	Loss: 0.137639
====> Epoch: 10 Average loss: 0.1038
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====

82
83
84
85
86
87
88
89
90
Train Epoch: 18 [450/504 (89%)]	Loss: 0.094114
91
92
93
94
95
96
97
98
99
100
Train Epoch: 18 [400/504 (99%)]	Loss: 0.135429
====> Epoch: 18 Average loss: 0.0996
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1161
0
Train Epoch: 19 [0/504 (0%)]	Loss: 0.094360
1
2
3
4
5
6
7
8
9
10
Train Epoch: 19 [50/504 (10%)]	Loss: 0.099923
11
12
13
14
15
16
17
18
19
20
Train Epoch: 19 [100/504 (20%)]	Loss: 0.080072
21
22
23
24
25
26
27
28
29
30
Train Epoch: 19 [150/504 (30%)]	Loss: 0.092631
31
32
33
34
35
36
37
38
39
40
Train Epoch: 19 [200/504 (40%)]	Loss: 0.097161
41
42
43
44
45
46
47
48
49
50
Train Epoch: 19 [250/504 (50%)]	Loss: 0.108707
51
52
53
54
55
56
57
58
59
60
Train Epoch: 19 [300/504 (59%)]	Loss: 0.113488
61
62
63
64
65
66
67
68
69
70
Train Epoch: 19 [350/504 (69%)]	Loss: 0.094018
71
72
73
74
75
76
77
78
79
80
Train Epoch: 19 [400/504 (79%)]	Loss: 0.097363
81
82
83
84
85
86
87
88
89
90
Train Epoch: 19 [450/504 (89%)]	Loss: 0

58
59
60
Train Epoch: 27 [300/504 (59%)]	Loss: 0.106028
61
62
63
64
65
66
67
68
69
70
Train Epoch: 27 [350/504 (69%)]	Loss: 0.114408
71
72
73
74
75
76
77
78
79
80
Train Epoch: 27 [400/504 (79%)]	Loss: 0.102111
81
82
83
84
85
86
87
88
89
90
Train Epoch: 27 [450/504 (89%)]	Loss: 0.097105
91
92
93
94
95
96
97
98
99
100
Train Epoch: 27 [400/504 (99%)]	Loss: 0.130805
====> Epoch: 27 Average loss: 0.0996
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1198
0
Train Epoch: 28 [0/504 (0%)]	Loss: 0.089884
1
2
3
4
5
6
7
8
9
10
Train Epoch: 28 [50/504 (10%)]	Loss: 0.101251
11
12
13
14
15
16
17
18
19
20
Train Epoch: 28 [100/504 (20%)]	Loss: 0.096657
21
22
23
24
25
26
27
28
29
30
Train Epoch: 28 [150/504 (30%)]	Loss: 0.105770
31
32
33
34
35
36
37
38
39
40
Train Epoch: 28 [200/504 (40%)]	Loss: 0.102062
41
42
43
44
45
46
47
48
49
50
Train Epoch: 28 [250/504 (50%)]	Loss: 0.093321
51
52
53
54
55
56
57
58
59
60
Train Epoch: 28 [300/504 (59%)]	Loss: 0.095749
61
62
63
6

32
33
34
35
36
37
38
39
40
Train Epoch: 36 [200/504 (40%)]	Loss: 0.088590
41
42
43
44
45
46
47
48
49
50
Train Epoch: 36 [250/504 (50%)]	Loss: 0.089282
51
52
53
54
55
56
57
58
59
60
Train Epoch: 36 [300/504 (59%)]	Loss: 0.098872
61
62
63
64
65
66
67
68
69
70
Train Epoch: 36 [350/504 (69%)]	Loss: 0.094041
71
72
73
74
75
76
77
78
79
80
Train Epoch: 36 [400/504 (79%)]	Loss: 0.090231
81
82
83
84
85
86
87
88
89
90
Train Epoch: 36 [450/504 (89%)]	Loss: 0.090057
91
92
93
94
95
96
97
98
99
100
Train Epoch: 36 [400/504 (99%)]	Loss: 0.111246
====> Epoch: 36 Average loss: 0.0936
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1198
0
Train Epoch: 37 [0/504 (0%)]	Loss: 0.088603
1
2
3
4
5
6
7
8
9
10
Train Epoch: 37 [50/504 (10%)]	Loss: 0.097710
11
12
13
14
15
16
17
18
19
20
Train Epoch: 37 [100/504 (20%)]	Loss: 0.101404
21
22
23
24
25
26
27
28
29
30
Train Epoch: 37 [150/504 (30%)]	Loss: 0.090081
31
32
33
34
35
36
37
38
39
40
Train Epoch: 37 [200/504 (40%)]	Loss: 0

...

82
83
84
85
86
87
88
89
90
Train Epoch: 62 [450/504 (89%)]	Loss: 0.088143
91
92
93
94
95
96
97
98
99
100
Train Epoch: 62 [400/504 (99%)]	Loss: 0.099926
====> Epoch: 62 Average loss: 0.0826
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1320
0
Train Epoch: 63 [0/504 (0%)]	Loss: 0.083912
1
2
3
4
5
6
7
8
9
10
Train Epoch: 63 [50/504 (10%)]	Loss: 0.083802
11
12
13
14
15
16
17
18
19
20
Train Epoch: 63 [100/504 (20%)]	Loss: 0.077127
21
22
23
24
25
26
27
28
29
30
Train Epoch: 63 [150/504 (30%)]	Loss: 0.098090
31
32
33
34
35
36
37
38
39
40
Train Epoch: 63 [200/504 (40%)]	Loss: 0.066727
41
42
43
44
45
46
47
48
49
50
Train Epoch: 63 [250/504 (50%)]	Loss: 0.085526
51
52
53
54
55
56
57
58
59
60
Train Epoch: 63 [300/504 (59%)]	Loss: 0.073398
61
62
63
64
65
66
67
68
69
70
Train Epoch: 63 [350/504 (69%)]	Loss: 0.068553
71
72
73
74
75
76
77
78
79
80
Train Epoch: 63 [400/504 (79%)]	Loss: 0.067750
81
82
83
84
85
86
87
88
89
90
Train Epoch: 63 [450/504 (89%)]	Loss: 0

59
60
Train Epoch: 71 [300/504 (59%)]	Loss: 0.088610
61
62
63
64
65
66
67
68
69
70
Train Epoch: 71 [350/504 (69%)]	Loss: 0.071705
71
72
73
74
75
76
77
78
79
80
Train Epoch: 71 [400/504 (79%)]	Loss: 0.079025
81
82
83
84
85
86
87
88
89
90
Train Epoch: 71 [450/504 (89%)]	Loss: 0.078727
91
92
93
94
95
96
97
98
99
100
Train Epoch: 71 [400/504 (99%)]	Loss: 0.127317
====> Epoch: 71 Average loss: 0.0805
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1367
0
Train Epoch: 72 [0/504 (0%)]	Loss: 0.089495
1
2
3
4
5
6
7
8
9
10
Train Epoch: 72 [50/504 (10%)]	Loss: 0.091385
11
12
13
14
15
16
17
18
19
20
Train Epoch: 72 [100/504 (20%)]	Loss: 0.073314
21
22
23
24
25
26
27
28
29
30
Train Epoch: 72 [150/504 (30%)]	Loss: 0.070606
31
32
33
34
35
36
37
38
39
40
Train Epoch: 72 [200/504 (40%)]	Loss: 0.074567
41
42
43
44
45
46
47
48
49
50
Train Epoch: 72 [250/504 (50%)]	Loss: 0.076901
51
52
53
54
55
56
57
58
59
60
Train Epoch: 72 [300/504 (59%)]	Loss: 0.085201
61
62
63
64
6

35
36
37
38
39
40
Train Epoch: 80 [200/504 (40%)]	Loss: 0.079196
41
42
43
44
45
46
47
48
49
50
Train Epoch: 80 [250/504 (50%)]	Loss: 0.081318
51
52
53
54
55
56
57
58
59
60
Train Epoch: 80 [300/504 (59%)]	Loss: 0.078705
61
62
63
64
65
66
67
68
69
70
Train Epoch: 80 [350/504 (69%)]	Loss: 0.086162
71
72
73
74
75
76
77
78
79
80
Train Epoch: 80 [400/504 (79%)]	Loss: 0.077872
81
82
83
84
85
86
87
88
89
90
Train Epoch: 80 [450/504 (89%)]	Loss: 0.083459
91
92
93
94
95
96
97
98
99
100
Train Epoch: 80 [400/504 (99%)]	Loss: 0.094894
====> Epoch: 80 Average loss: 0.0791
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1389
0
Train Epoch: 81 [0/504 (0%)]	Loss: 0.069855
1
2
3
4
5
6
7
8
9
10
Train Epoch: 81 [50/504 (10%)]	Loss: 0.079178
11
12
13
14
15
16
17
18
19
20
Train Epoch: 81 [100/504 (20%)]	Loss: 0.070669
21
22
23
24
25
26
27
28
29
30
Train Epoch: 81 [150/504 (30%)]	Loss: 0.082335
31
32
33
34
35
36
37
38
39
40
Train Epoch: 81 [200/504 (40%)]	Loss: 0.083479
4

13
14
15
16
17
18
19
20
Train Epoch: 89 [100/504 (20%)]	Loss: 0.071924
21
22
23
24
25
26
27
28
29
30
Train Epoch: 89 [150/504 (30%)]	Loss: 0.074173
31
32
33
34
35
36
37
38
39
40
Train Epoch: 89 [200/504 (40%)]	Loss: 0.074925
41
42
43
44
45
46
47
48
49
50
Train Epoch: 89 [250/504 (50%)]	Loss: 0.083038
51
52
53
54
55
56
57
58
59
60
Train Epoch: 89 [300/504 (59%)]	Loss: 0.066642
61
62
63
64
65
66
67
68
69
70
Train Epoch: 89 [350/504 (69%)]	Loss: 0.059860
71
72
73
74
75
76
77
78
79
80
Train Epoch: 89 [400/504 (79%)]	Loss: 0.073996
81
82
83
84
85
86
87
88
89
90
Train Epoch: 89 [450/504 (89%)]	Loss: 0.083571
91
92
93
94
95
96
97
98
99
100
Train Epoch: 89 [400/504 (99%)]	Loss: 0.110954
====> Epoch: 89 Average loss: 0.0769
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1385
0
Train Epoch: 90 [0/504 (0%)]	Loss: 0.088392
1
2
3
4
5
6
7
8
9
10
Train Epoch: 90 [50/504 (10%)]	Loss: 0.086049
11
12
13
14
15
16
17
18
19
20
Train Epoch: 90 [100/504 (20%)]	Loss: 0.08

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====> Test set loss: 0.1359
0
Train Epoch: 98 [0/504 (0%)]	Loss: 0.081471
1
2
3
4
5
6
7
8
9
10
Train Epoch: 98 [50/504 (10%)]	Loss: 0.080168
11
12
13
14
15
16
17
18
19
20
Train Epoch: 98 [100/504 (20%)]	Loss: 0.088946
21
22
23
24
25
26
27
28
29
30
Train Epoch: 98 [150/504 (30%)]	Loss: 0.071904
31
32
33
34
35
36
37
38
39
40
Train Epoch: 98 [200/504 (40%)]	Loss: 0.072407
41
42
43
44
45
46
47
48
49
50
Train Epoch: 98 [250/504 (50%)]	Loss: 0.080013
51
52
53
54
55
56
57
58
59
60
Train Epoch: 98 [300/504 (59%)]	Loss: 0.070687
61
62
63
64
65
66
67
68
69
70
Train Epoch: 98 [350/504 (69%)]	Loss: 0.092155
71
72
73
74
75
76
77
78
79
80
Train Epoch: 98 [400/504 (79%)]	Loss: 0.081567
81
82
83
84
85
86
87
88
89
90
Train Epoch: 98 [450/504 (89%)]	Loss: 0.082812
91
92
93
94
95
96
97
98
99
100
Train Epoch: 98 [400/504 (99%)]	Loss: 0.083107
====> Epoch: 98 Average loss: 0.0784
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
====

In [11]:
input_data.shape

(504, 548, 421)